In [74]:
import pandas as pd
from pyspark.sql import functions as F
import numpy as np

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/12 14:34:30 WARN Utils: Your hostname, DESKTOP-1D7SN6N resolves to a loopback address: 127.0.1.1; using 172.23.1.215 instead (on interface eth0)
22/09/12 14:34:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 14:34:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [93]:
properties = pd.read_csv('../data/curated/properties.csv') 
properties = properties.reset_index().dropna(subset = ['weekly_cost'])

In [94]:
schools = pd.read_csv('../data/curated/schools.csv')

In [95]:
sparkProperty=spark.createDataFrame(properties) 
sparkSchool = spark.createDataFrame(schools)

In [97]:
#https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula

from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...
distanceUDF = F.udf(lambda a,b,c,d: distance(a,b,c,d))

In [98]:
# For each property, calculate its distance to every school
# then count for number of schools within 3km of each property

school_count = sparkProperty \
            .crossJoin(sparkSchool) \
            .withColumn("distance", 
                        distanceUDF(F.col("prop_lat"), F.col("prop_long"), F.col("school_lat"), F.col("school_long"))
                       ) \
            .filter(F.col("distance") <= 3) \
            .groupBy(F.col("index")) \
            .count() \
            .join(sparkProperty, 'index', 'right') \
            .orderBy('index') \
            .toPandas()

22/09/12 15:03:10 WARN ExtractPythonUDFFromJoinCondition: The join condition:(cast(<lambda>(prop_lat#1067, prop_long#1068, school_lat#1085, school_long#1084)#1100 as int) <= 3) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


In [99]:
# Fill propertys having no schools within 3km as 0
school_count[['count']] = school_count[['count']].fillna(0)

In [106]:
school_count = school_count.rename(columns = {'count':'numSchool_3km'})

,index,numSchool_3km,name,type,prop_lat,prop_long,Beds,Baths,Parking,weekly_cost,suburb
0,0,33.0,56 Selwyn Street Albion VIC 3020,House,-37.780782,144.813020,3,2,2,300.0,3020
1,1,12.0,1/3 Robin Street Altona VIC 3018,Apartment / Unit / Flat,-37.869028,144.807250,2,1,1,300.0,3018
2,2,6.0,34 Hucker Street Ararat VIC 3377,House,-37.276411,142.920079,3,1,1,300.0,3377
3,3,6.0,3 Neylan Street Ararat VIC 3377,House,-37.292905,142.921881,2,1,2,300.0,3377
4,4,30.0,8 Suspension Street Ardeer VIC 3022,House,-37.782361,144.808833,2,1,2,300.0,3022
...,...,...,...,...,...,...,...,...,...,...,...
9389,9666,15.0,31 Austin Road Seaford VIC 3198,House,-38.105231,145.134350,4,2,2,850.0,3198
9390,9667,2.0,12 Morobe Street Sorrento VIC 3943,House,-38.354365,144.758948,5,2,0,850.0,3943
9391,9668,50.0,7/278 Kings Way South Melbourne VIC 3205,Townhouse,-37.833083,144.966977,3,2,1,850.0,3205
9392,9669,58.0,37 Cunningham Street South Yarra VIC 3141,House,-37.840129,144.996587,3,1,1,850.0,3141
